# Testing on Colab

These are all the imports you will need. If you encounter any errors on your local device, use pip install "module_name" -U to update it. For instance, the older version of pinecone-client differs from the new one, so the pinecone component will not work if you're using the outdated version.

In [1]:
%pip install langchain --quiet
%pip install accelerate --quiet
%pip install transformers --quiet
%pip install bitsandbytes --quiet
%pip install unstructured --quiet
%pip install sentence-transformers --quiet
%pip install -U faiss-gpu tiktoken
%pip install langchainhub --quiet
%pip install pinecone-client --quiet
%pip install langchain_pinecone --quiet
%pip install rank_bm25 --quiet
%pip install python-dotenv --quiet



In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import yaml
with open("cfg.yaml", "r") as file:
    cfg = yaml.safe_load(file)

please leave the local_path empty and
comment in either faiss or pinecone part in retrievers
(if bm25, faiss and pinecone are all commented out in the cfg.yaml, than it will fail.
you need [bm25] + [faiss or pinecone] for ensemble retriever which we use)

In [4]:
cfg

{'retrievers': {'bm25': {'topk': 4, 'path': 'chunked_docs.pkl'},
  'faiss': {'faiss_index_path': 'faiss_indices/', 'topk': 4}},
 'ensemble': {'weights': [0.4, 0.6]},
 'llm_model': {'name': 'mistralai/Mistral-7B-Instruct-v0.1',
  'local_path': '',
  'temperature': 0.001,
  'top_p': 0.95,
  'repetition_penalty': 1.15},
 'embedding_model': 'thenlper/gte-base',
 'use_qa_chain': True,
 'use_conversational_qa_chain': True,
 'use_multi_query_qa_chain': True}

In [5]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


Make sure chunked_docs.pkl and the faiss_indices/thenlper/gte-base are located in the current working directory. 

You don't need faiss_indices/.. if you are using Pinecone.

In [6]:
from custom_chatbot import MedicalChatbot

cuda is available: True
mps is available: False


In [7]:
chatbot = MedicalChatbot(cfg)


generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

The Conversational Retrieval Chain works successfully on Colab.

You can also use chatbot.generate_response_by_type(query, type='conversational') instead chatbot.generate_response_with_conversational(query)

In [8]:
query = "Is there a cure for any type of cancer?"
chatbot.generate_response_with_conversational(query)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Abstract: The goal of all medical activity is to preserve health in fit people and to restore the sick into a state of complete physical mental and social wellbeing In an effort to determine whether we are achieving this last goal in oncology herein we review the biological and clinical framework that has led to the foundations of the current anticancer treatment paradigm Currently cancer therapy is still based on the ancient axiom that states that the complete eradication of the tumor burden is the only way to achieve a cure This strategy has led to a substantial improvement in survival rates as cancer mortality rates have dropped in an unprecedented way Despite this progress more than 9 million people still die from ca

{'question': 'Is there a cure for any type of cancer?',
 'chat_history': [],
 'answer': ' No, currently there is no cure for any type of cancer. While cancer therapy has improved substantially over time, resulting in a significant decrease in cancer mortality rates, it does not lead to a cancer cure but rather to a cancer remission.'}

In [9]:
query = "Do you think we will find a cure for it in the near future?"
chatbot.generate_response_with_conversational(query)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Is there a cure for any type of cancer?
Assistant:  No, currently there is no cure for any type of cancer. While cancer therapy has improved substantially over time, resulting in a significant decrease in cancer mortality rates, it does not lead to a cancer cure but rather to a cancer remission.
Follow Up Input: Do you think we will find a cure for it in the near future?
Standalone question:

> Finished chain.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Abstract: The goal of all medical activity is to preserve health in fit people and to restore the sick into a state of complete physical mental and social wellbeing In an effort to determine whether we are achieving this last goal in oncology herein we review the biological and clinical framework that has led to the foundations of the current anticancer treatment paradigm Currently cancer therapy is still based on the ancient axiom that states that the complete eradication of the tumor burden is the only way to achieve a cure This strategy has led to a substantial improvement in survival rates as cancer mortality rates have dropped in an unprecedented way Despite this progress more than 9 million people still die from ca

{'question': 'Do you think we will find a cure for it in the near future?',
 'chat_history': [('Is there a cure for any type of cancer?',
   ' No, currently there is no cure for any type of cancer. While cancer therapy has improved substantially over time, resulting in a significant decrease in cancer mortality rates, it does not lead to a cancer cure but rather to a cancer remission.')],
 'answer': ' Based on the provided abstracts, there are several advancements in cancer research that are showing promise for a cure in the near future. These include gene editing using CRISPR-Cas9, theranostics (the use of diagnostic tests to guide treatment), viral mediated therapy, artificial intelligence, tumor infiltrating lymphocyte therapy, nanoparticles, gene therapy, biomarkers, and immunotherapy. Additionally, there is ongoing research in areas such as cancer stem cells, molecularly targeted therapies, genomic analyses, and individually tailored immunotherapy. While a cure for cancer may not 

In [10]:
query = "How many years do you think we need to wait for it?"
chatbot.generate_response_with_conversational(query)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Is there a cure for any type of cancer?
Assistant:  No, currently there is no cure for any type of cancer. While cancer therapy has improved substantially over time, resulting in a significant decrease in cancer mortality rates, it does not lead to a cancer cure but rather to a cancer remission.
Human: Do you think we will find a cure for it in the near future?
Assistant:  Based on the provided abstracts, there are several advancements in cancer research that are showing promise for a cure in the near future. These include gene editing using CRISPR-Cas9, theranostics (the use of diagnostic tests to guide treatment), viral mediated therapy, artificial intelligence, tumor infiltrating lymphocyte therapy, nanoparticles, gene therapy, biomarkers, and immunotherapy.

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Abstract: The goal of all medical activity is to preserve health in fit people and to restore the sick into a state of complete physical mental and social wellbeing In an effort to determine whether we are achieving this last goal in oncology herein we review the biological and clinical framework that has led to the foundations of the current anticancer treatment paradigm Currently cancer therapy is still based on the ancient axiom that states that the complete eradication of the tumor burden is the only way to achieve a cure This strategy has led to a substantial improvement in survival rates as cancer mortality rates have dropped in an unprecedented way Despite this progress more than 9 million people still die from ca

{'question': 'How many years do you think we need to wait for it?',
 'chat_history': [('Is there a cure for any type of cancer?',
   ' No, currently there is no cure for any type of cancer. While cancer therapy has improved substantially over time, resulting in a significant decrease in cancer mortality rates, it does not lead to a cancer cure but rather to a cancer remission.'),
  ('Do you think we will find a cure for it in the near future?',
   ' Based on the provided abstracts, there are several advancements in cancer research that are showing promise for a cure in the near future. These include gene editing using CRISPR-Cas9, theranostics (the use of diagnostic tests to guide treatment), viral mediated therapy, artificial intelligence, tumor infiltrating lymphocyte therapy, nanoparticles, gene therapy, biomarkers, and immunotherapy. Additionally, there is ongoing research in areas such as cancer stem cells, molecularly targeted therapies, genomic analyses, and individually tailore

The Basic Retriaval QA Chain works successfully on Colab.

You can also use chatbot.generate_response_by_type(query, type='basic') instead chatbot.generate_response(query)

In [11]:
query = "Is there a cure for any type of cancer?"
chatbot.generate_response(query)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'query': 'Is there a cure for any type of cancer?',
 'result': ' Based on the given context, I cannot provide a definitive answer to the question "Is there a cure for any type of cancer?" However, it does mention that current cancer therapy is still based on the ancient axiom that states that the complete eradication of the tumor burden is the only way to achieve a cure. This strategy has led to a substantial improvement in survival rates, but more than 9 million people still die from cancer every year, indicating that the current treatment strategy is not leading to a cancer cure but to a cancer remission that is the temporary absence of manifestations of a particular disease after months or years of remission in most patients. The article suggests that it is time to discuss about the new key challenges and future directions in clinical oncology and generate novel treatment strategies more suited to the current clinical reality.',
 'source_documents': [Document(page_content='Abstract

The Multi-Query Retrieval Chain works successfully on Colab.

You can also use chatbot.generate_response_by_type(query, type='multi_query') instead chatbot.generate_response(query)

In [12]:
query = "Is there a cure for any type of cancer?"
chatbot.generate_response_with_multi_query(query)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
INFO:langchain.retrievers.multi_query:Generated queries: ['Version 1: Can you provide information about treatments or cures for various types of cancer?', '    Version 2: Are there any known remedies or therapies that can effectively treat cancer?', '    Version 3: Could you tell me if there are any available treatments or interventions for different types of cancer?']
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'question': 'Is there a cure for any type of cancer?',
 'query': 'Is there a cure for any type of cancer?',
 'context': '232 7 rhetoric pertaining to the latest technology n63 203 and 8 focusing treatment on cancer origins and cause n43 139 Overall 258 n80 of advertisements included a direct statement claiming provider treatment can cure cancer or prolong life Our results provide evidence alternative cancer providers are using Meta advertising products to market scientifically unsupported cancer treatments Advertisements regularly referenced alternative and natural treatment approaches to cancer Imagery and text content that emulated evidencebased medical providers created the impression that the offered treatments were effective medical options for cancer Advertisements exploited the hope of patients with terminal and poor prognoses by sharing testimonials of past patients who allegedly were cured or had their lives prolonged We recommend that Meta introduce a mandatory humanled auth

# Testing on a mac with a model downloaded using GPT4ALL

Be sure you made all the import at the top of the page. If you encounter any errors on your local device, use pip install "module_name" -U to update it.

Make sure chunked_docs.pkl and the faiss_indices/thenlper/gte-base are located in the current working directory. 

You don't need faiss_indices/.. if you are using Pinecone.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import yaml
with open("cfg.yaml", "r") as file:
    cfg = yaml.safe_load(file)

Add a local path to load a model downloaded using GPT4ALL for improved performance on a Mac with an M-chip.

In [3]:
cfg

{'retrievers': {'bm25': {'topk': 4, 'path': 'chunked_docs.pkl'},
  'faiss': {'faiss_index_path': 'faiss_indices/', 'topk': 4}},
 'ensemble': {'weights': [0.4, 0.6]},
 'llm_model': {'name': 'GPT4ALL Falcon 7B',
  'local_path': '/Users/yusuf/Library/Application Support/nomic.ai/GPT4All/gpt4all-falcon-newbpe-q4_0.gguf',
  'temperature': 0.001,
  'top_p': 0.95,
  'repetition_penalty': 1.15},
 'embedding_model': 'thenlper/gte-base',
 'use_qa_chain': True,
 'use_conversational_qa_chain': True,
 'use_multi_query_qa_chain': True}

In [4]:
from custom_chatbot import MedicalChatbot

cuda is available: False
mps is available: True


In [5]:
chatbot = MedicalChatbot(cfg)

The Conversational Retrieval Chain operates successfully on a Mac with a model loaded using GPT4ALL.

You can also use chatbot.generate_response_by_type(query, type='basic') instead chatbot.generate_response(query)

It saves the last two question-answer pairs

In [6]:
query = "Is there a cure for any type of cancer?"
chatbot.generate_response_with_conversational(query)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Abstract: The goal of all medical activity is to preserve health in fit people and to restore the sick into a state of complete physical mental and social wellbeing In an effort to determine whether we are achieving this last goal in oncology herein we review the biological and clinical framework that has led to the foundations of the current anticancer treatment paradigm Currently cancer therapy is still based on the ancient axiom that states that the complete eradication of the tumor burden is the only way to achieve a cure This strategy has led to a substantial improvement in survival rates as cancer mortality rates have dropped in an unprecedented way Despite this progress more than 9 million people still die from ca

{'question': 'Is there a cure for any type of cancer?',
 'chat_history': [],
 'answer': " No, there is no cure for cancer. However, with advancements in technology and research, treatments are becoming more effective and personalized to each individual's needs."}

In [7]:
query = "Do you think we will find a cure for it in the near future?"
chatbot.generate_response_with_conversational(query)



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Is there a cure for any type of cancer?
Assistant:  No, there is no cure for cancer. However, with advancements in technology and research, treatments are becoming more effective and personalized to each individual's needs.
Follow Up Input: Do you think we will find a cure for it in the near future?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Abstract: The goal of all medical activity is to preserve health in fit people and to restore the sick into a state of complete physical mental and social wellb

{'question': 'Do you think we will find a cure for it in the near future?',
 'chat_history': [('Is there a cure for any type of cancer?',
   " No, there is no cure for cancer. However, with advancements in technology and research, treatments are becoming more effective and personalized to each individual's needs.")],
 'answer': '  While there is no guarantee that we will find a cure for cancer in the near future, there are many promising developments in cancer research and treatment. With continued advancements in technology and increased funding for cancer research, it is possible that we may see significant progress in the fight against this disease in the coming years. However, it is important to remember that cancer is a complex disease with many different causes and treatments, and finding a cure will require ongoing research and collaboration between medical professionals, researchers, and patients themselves.'}

In [8]:
query = "How many years do you think we need to wait for it?"
chatbot.generate_response_with_conversational(query)



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Is there a cure for any type of cancer?
Assistant:  No, there is no cure for cancer. However, with advancements in technology and research, treatments are becoming more effective and personalized to each individual's needs.
Human: Do you think we will find a cure for it in the near future?
Assistant:   While there is no guarantee that we will find a cure for cancer in the near future, there are many promising developments in cancer research and treatment. With continued advancements in technology and increased funding for cancer research, it is possible that we may see significant progress in the fight against this disease in the coming years. However, it is important to remember that cancer is a complex disease with many different causes and treatments, and fi

{'question': 'How many years do you think we need to wait for it?',
 'chat_history': [('Is there a cure for any type of cancer?',
   " No, there is no cure for cancer. However, with advancements in technology and research, treatments are becoming more effective and personalized to each individual's needs."),
  ('Do you think we will find a cure for it in the near future?',
   '  While there is no guarantee that we will find a cure for cancer in the near future, there are many promising developments in cancer research and treatment. With continued advancements in technology and increased funding for cancer research, it is possible that we may see significant progress in the fight against this disease in the coming years. However, it is important to remember that cancer is a complex disease with many different causes and treatments, and finding a cure will require ongoing research and collaboration between medical professionals, researchers, and patients themselves.')],
 'answer': '  Fin

The Basic Retrieval QA Chain operates successfully on a Mac with a model loaded using GPT4ALL.

You can also use chatbot.generate_response_by_type(query, type='basic') instead chatbot.generate_response(query)

In [9]:
query = "Is there a cure for any type of cancer?"
chatbot.generate_response(query)

{'query': 'Is there a cure for any type of cancer?',
 'result': "\n\nAbstract_Length: 12, 203 words\n\nAbstract_Summary: The study aimed to understand primary caregivers' perceptions and concerns about sexuality and reproductive health of individuals with intellectual disability (IDD) in India. A qualitative methodology was adopted, and 14 primary caregivers were interviewed in Mumbai. The common perceptions were that puberty is expected, sexual behavior is unexpected, and there is a hope for cure. Puberty onset was often seen as an enabling factor for marriage, especially among parents of female children. Marriage and childbirth were perceived as possible cures for IDD by some parents. The dominant parental concerns were found to be safety, early onset of puberty, perception of child's action by others, and concerns about the child's family life. Some of the perceptions about sexuality were shaped by the primary caregivers' concern for the individual with IDD. Three patients with IS w

The Multi-Query Retrieval Chain operates successfully on a Mac with a model loaded using GPT4ALL.

You can also use chatbot.generate_response_by_type(query, type='multi_query') instead chatbot.generate_response(query)

In [10]:
query = "Is there a cure for any type of cancer?"
chatbot.generate_response_with_multi_query(query)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What are the current treatments available for cancer?', '2. How effective are these treatments in curing cancer?', '3. Are there any potential side effects or risks associated with these treatments?']


{'question': 'Is there a cure for any type of cancer?',
 'query': 'Is there a cure for any type of cancer?',
 'context': 'Abstract: The goal of all medical activity is to preserve health in fit people and to restore the sick into a state of complete physical mental and social wellbeing In an effort to determine whether we are achieving this last goal in oncology herein we review the biological and clinical framework that has led to the foundations of the current anticancer treatment paradigm Currently cancer therapy is still based on the ancient axiom that states that the complete eradication of the tumor burden is the only way to achieve a cure This strategy has led to a substantial improvement in survival rates as cancer mortality rates have dropped in an unprecedented way Despite this progress more than 9 million people still die from cancer every year indicating that the current treatment strategy is not leading to a cancer cure but to a cancer remission that is the temporary absen